In [1]:
import pandas as pd
import requests
import json

* 2023.01 ~ 2023.11 까지 지하철 승하차 데이터

In [8]:
# 컬럼명 다 보이게 하기
pd.set_option('display.max_seq_items', None)

In [39]:
d = pd.read_csv('./CARD_SUBWAY_MONTH_202301.csv')
d.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18797 entries, 20230101 to 20230131
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   사용일자    18797 non-null  object 
 1   노선명     18797 non-null  object 
 2   역명      18797 non-null  int64  
 3   승차총승객수  18797 non-null  int64  
 4   하차총승객수  18797 non-null  int64  
 5   등록일자    0 non-null      float64
dtypes: float64(1), int64(3), object(2)
memory usage: 1.0+ MB


In [40]:
d.columns

Index(['사용일자', '노선명', '역명', '승차총승객수', '하차총승객수', '등록일자'], dtype='object')

In [58]:
d = d.reset_index()

In [63]:
d = d.rename(columns={"index":"등록일자1"})

In [66]:
d = d.drop("등록일자",axis=1)

In [69]:
d = d.rename(columns={"등록일자1":"사용일자","사용일자":"노선명","노선명":"역명","역명":"승차총승객수","승차총승객수":"하차총승객수"})

In [70]:
d

,사용일자,노선명,역명,승차총승객수,하차총승객수,하차총승객수
0,20230101,경강선,곤지암,1450,1539,20230104
1,20230101,2호선,신도림,20210,21037,20230104
2,20230101,2호선,문래,5995,6567,20230104
3,20230101,2호선,영등포구청,5629,6117,20230104
4,20230101,2호선,당산,6821,7594,20230104
...,...,...,...,...,...,...
18792,20230131,경원선,소요산,2244,1929,20230203
18793,20230131,경원선,동두천,1988,2188,20230203
18794,20230131,경원선,보산,1391,1299,20230203
18795,20230131,경원선,동두천중앙,3046,2954,20230203


In [54]:
d['사용일자'].str.split("").str[0]

20230101     
20230101     
20230101     
20230101     
20230101     
           ..
20230131     
20230131     
20230131     
20230131     
20230131     
Name: 사용일자, Length: 18797, dtype: object

In [74]:
# 월별 지하철 승하차 정보 불러오고 합치기

combined_data = pd.DataFrame()

# 파일을 순회하며 데이터 합치기
for i in range(202301, 202312):
    file_name = f'CARD_SUBWAY_MONTH_{i}.csv'
    try:
        # CSV 파일 읽기
        data = pd.read_csv(file_name)
        data = data.reset_index()
        
        # 읽어온 데이터를 기존 데이터에 추가
        combined_data = pd.concat([combined_data, data], ignore_index=True)
        print(f"File {file_name} added.")
    except FileNotFoundError:
        print(f"File {file_name} not found.")
    except Exception as e:
        print(f"An error occurred while processing {file_name}: {str(e)}")


File CARD_SUBWAY_MONTH_202301.csv added.
File CARD_SUBWAY_MONTH_202302.csv added.
File CARD_SUBWAY_MONTH_202303.csv added.
File CARD_SUBWAY_MONTH_202304.csv added.
File CARD_SUBWAY_MONTH_202305.csv added.
File CARD_SUBWAY_MONTH_202306.csv added.
File CARD_SUBWAY_MONTH_202307.csv added.
File CARD_SUBWAY_MONTH_202308.csv added.
File CARD_SUBWAY_MONTH_202309.csv added.
File CARD_SUBWAY_MONTH_202310.csv added.
File CARD_SUBWAY_MONTH_202311.csv added.


In [80]:
combined_data = combined_data.rename(columns={"index":"등록일자1"})

In [83]:
combined_data = combined_data.drop("등록일자",axis=1)

In [84]:
combined_data

,등록일자1,사용일자,노선명,역명,승차총승객수,하차총승객수
0,20230101,경강선,곤지암,1450,1539,20230104
1,20230101,2호선,신도림,20210,21037,20230104
2,20230101,2호선,문래,5995,6567,20230104
3,20230101,2호선,영등포구청,5629,6117,20230104
4,20230101,2호선,당산,6821,7594,20230104
...,...,...,...,...,...,...
203201,20231130,5호선,발산,24637,25129,20231203
203202,20231130,5호선,우장산,16074,16027,20231203
203203,20231130,5호선,화곡,31199,29241,20231203
203204,20231130,5호선,까치산,30082,28933,20231203


In [86]:
combined_data = combined_data.rename(columns={"등록일자1":"사용일자","사용일자":"노선명","노선명":"역명","역명":"승차총승객수","승차총승객수":"하차총승객수","하차총승객수":"등록일자"})      

In [87]:
combined_data

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,20230101,경강선,곤지암,1450,1539,20230104
1,20230101,2호선,신도림,20210,21037,20230104
2,20230101,2호선,문래,5995,6567,20230104
3,20230101,2호선,영등포구청,5629,6117,20230104
4,20230101,2호선,당산,6821,7594,20230104
...,...,...,...,...,...,...
203201,20231130,5호선,발산,24637,25129,20231203
203202,20231130,5호선,우장산,16074,16027,20231203
203203,20231130,5호선,화곡,31199,29241,20231203
203204,20231130,5호선,까치산,30082,28933,20231203


In [19]:
subway_data = pd.read_csv('combined_subway_data_202301_to_202311.csv')

In [20]:
subway_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203206 entries, 0 to 203205
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   사용일자    203206 non-null  object 
 1   노선명     203206 non-null  object 
 2   역명      203206 non-null  int64  
 3   승차총승객수  203206 non-null  int64  
 4   하차총승객수  203206 non-null  int64  
 5   등록일자    0 non-null       float64
dtypes: float64(1), int64(3), object(2)
memory usage: 9.3+ MB


In [21]:
subway_data.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,경강선,곤지암,1450,1539,20230104,NaN
1,2호선,신도림,20210,21037,20230104,NaN
2,2호선,문래,5995,6567,20230104,NaN
3,2호선,영등포구청,5629,6117,20230104,NaN
4,2호선,당산,6821,7594,20230104,NaN


In [22]:
subway_data.columns

Index(['사용일자', '노선명', '역명', '승차총승객수', '하차총승객수', '등록일자'], dtype='object')

In [23]:
subway_data.drop(columns=['등록일자'], inplace=True)

In [25]:
subway_data['사용일자']

0         경강선
1         2호선
2         2호선
3         2호선
4         2호선
         ... 
203201    5호선
203202    5호선
203203    5호선
203204    5호선
203205    5호선
Name: 사용일자, Length: 203206, dtype: object

* 2023.01 ~ 2023.11 날씨 데이터

In [9]:
# 날씨 데이터 오픈 api 가져오기
url = 'https://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'

In [ ]:
# 서비스 키 에러나면 이걸로 다시
# CYfhWwsG3T6CbKfbZn%2BoWe5Ks2hKiIksiycfndf9OYy2l2XlQL4s1y1L5kDlVqe0JzyiTTRcsSO8Np03ikBXXA%3D%3D 

In [16]:
payload = dict(serviceKey='CYfhWwsG3T6CbKfbZn+oWe5Ks2hKiIksiycfndf9OYy2l2XlQL4s1y1L5kDlVqe0JzyiTTRcsSO8Np03ikBXXA==',
              pageNo='1',
               numOfRows='100',
               dataType = 'JSON',
               dateCd = 'HR',
               dataCd = 'ASOS',
               startDt = '20230101',
               startHh = '01',
               endDt = '20231130',
               endHh = '01',
               stnIds = '108'
              )

In [17]:
r = requests.get(url, params=payload)

In [18]:
r.text

'{"response":{"header":{"resultCode":"00","resultMsg":"NORMAL_SERVICE"},"body":{"dataType":"JSON","items":{"item":[{"tm":"2023-01-01 01:00","rnum":"1","stnId":"108","stnNm":"서울","ta":"1.5","taQcflg":"","rn":"","rnQcflg":"","ws":"1.9","wsQcflg":"","wd":"250","wdQcflg":"","hm":"71","hmQcflg":"","pv":"4.9","td":"-3.1","pa":"1018.0","paQcflg":"","ps":"1028.9","psQcflg":"","ss":"","ssQcflg":"9","icsr":"","dsnw":"","hr3Fhsc":"","dc10Tca":"6","dc10LmcsCa":"6","clfmAbbrCd":"","lcsCh":"12","vs":"1104","gndSttCd":"","dmstMtphNo":"","ts":"-1.4","tsQcflg":"","m005Te":"-0.3","m01Te":"-0.5","m02Te":"-0.5","m03Te":"0.6"},{"tm":"2023-01-01 02:00","rnum":"2","stnId":"108","stnNm":"서울","ta":"1.5","taQcflg":"","rn":"","rnQcflg":"","ws":"1.9","wsQcflg":"","wd":"250","wdQcflg":"","hm":"72","hmQcflg":"","pv":"4.9","td":"-3.0","pa":"1017.8","paQcflg":"","ps":"1028.7","psQcflg":"","ss":"","ssQcflg":"9","icsr":"","dsnw":"","hr3Fhsc":"","dc10Tca":"9","dc10LmcsCa":"8","clfmAbbrCd":"","lcsCh":"11","vs":"925","gnd

In [19]:
# 데이터 컬럼 찾고 변수에 담기
a = list(json.loads(r.text)['response']['body']['items']['item'][0].keys())

In [20]:
a

['tm',
 'rnum',
 'stnId',
 'stnNm',
 'ta',
 'taQcflg',
 'rn',
 'rnQcflg',
 'ws',
 'wsQcflg',
 'wd',
 'wdQcflg',
 'hm',
 'hmQcflg',
 'pv',
 'td',
 'pa',
 'paQcflg',
 'ps',
 'psQcflg',
 'ss',
 'ssQcflg',
 'icsr',
 'dsnw',
 'hr3Fhsc',
 'dc10Tca',
 'dc10LmcsCa',
 'clfmAbbrCd',
 'lcsCh',
 'vs',
 'gndSttCd',
 'dmstMtphNo',
 'ts',
 'tsQcflg',
 'm005Te',
 'm01Te',
 'm02Te',
 'm03Te']

In [21]:
for i  in json.loads(r.text)["response"]['body']['items']['item']:
    print(list(i.values()))

['2023-01-01 01:00', '1', '108', '서울', '1.5', '', '', '', '1.9', '', '250', '', '71', '', '4.9', '-3.1', '1018.0', '', '1028.9', '', '', '9', '', '', '', '6', '6', '', '12', '1104', '', '', '-1.4', '', '-0.3', '-0.5', '-0.5', '0.6']
['2023-01-01 02:00', '2', '108', '서울', '1.5', '', '', '', '1.9', '', '250', '', '72', '', '4.9', '-3.0', '1017.8', '', '1028.7', '', '', '9', '', '', '', '9', '8', '', '11', '925', '', '40', '-1.6', '', '-0.3', '-0.5', '-0.5', '0.6']
['2023-01-01 03:00', '3', '108', '서울', '1.6', '', '', '', '1.6', '', '250', '', '74', '', '5.1', '-2.5', '1018.1', '', '1029.0', '', '', '9', '', '', '', '8', '8', 'Sc', '16', '794', '', '40', '-1.8', '', '-0.3', '-0.5', '-0.5', '0.6']
['2023-01-01 04:00', '4', '108', '서울', '1.5', '', '', '', '1.4', '', '290', '', '74', '', '5.0', '-2.6', '1018.1', '', '1029.0', '', '', '9', '', '', '', '9', '9', 'Sc', '10', '709', '', '40', '-1.3', '', '-0.2', '-0.5', '-0.5', '0.6']
['2023-01-01 05:00', '5', '108', '서울', '0.8', '', '', '', '2.

In [23]:
weather_list = []

for i in range(1,74):
    url = f'https://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?serviceKey=CYfhWwsG3T6CbKfbZn%2BoWe5Ks2hKiIksiycfndf9OYy2l2XlQL4s1y1L5kDlVqe0JzyiTTRcsSO8Np03ikBXXA%3D%3D&pageNo={i}&numOfRows=100&dataType=JSON&dataCd=ASOS&dateCd=HR&startDt=20230101&startHh=01&endDt=20231130&endHh=01&stnIds=108'
    r = requests.get(url)

    for j in json.loads(r.text)['response']['body']['items']['item']:
        info = list(j.values())
        weather_list.append(info)

In [9]:
pd.DataFrame(weather_list,columns=a)

NameError: name 'weather_list' is not defined

In [25]:
weather_info = pd.DataFrame(weather_list,columns=a)

In [26]:
# CSV 파일로 저장
weather_info.to_csv('weather_data.csv', index=True)  # index=False를 지정하면 인덱스를 저장하지 않음

In [4]:
# 데이터 불러오기
weather_data = pd.read_csv('./weather_data.csv')

In [10]:
weather_data

,Unnamed: 0,tm,rnum,stnId,stnNm,ta,taQcflg,rn,rnQcflg,ws,...,lcsCh,vs,gndSttCd,dmstMtphNo,ts,tsQcflg,m005Te,m01Te,m02Te,m03Te
0,0,2023-01-01 01:00,1,108,서울,1.5,NaN,NaN,NaN,1.9,...,12.0,1104,NaN,NaN,-1.4,NaN,-0.3,-0.5,-0.5,0.6
1,1,2023-01-01 02:00,2,108,서울,1.5,NaN,NaN,NaN,1.9,...,11.0,925,NaN,40.0,-1.6,NaN,-0.3,-0.5,-0.5,0.6
2,2,2023-01-01 03:00,3,108,서울,1.6,NaN,NaN,NaN,1.6,...,16.0,794,NaN,40.0,-1.8,NaN,-0.3,-0.5,-0.5,0.6
3,3,2023-01-01 04:00,4,108,서울,1.5,NaN,NaN,NaN,1.4,...,10.0,709,NaN,40.0,-1.3,NaN,-0.2,-0.5,-0.5,0.6
4,4,2023-01-01 05:00,5,108,서울,0.8,NaN,NaN,NaN,2.4,...,11.0,1185,NaN,40.0,-2.1,NaN,-0.2,-0.5,-0.5,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7295,7295,2023-11-01 00:00,7296,108,서울,17.4,NaN,NaN,NaN,1.0,...,4.0,830,NaN,4019.0,15.6,NaN,15.7,16.4,16.3,16.5
7296,7296,2023-11-01 01:00,7297,108,서울,17.4,NaN,NaN,NaN,1.3,...,4.0,824,NaN,19.0,15.7,NaN,15.8,16.4,16.2,16.5
7297,7297,2023-11-01 02:00,7298,108,서울,17.6,NaN,NaN,NaN,2.9,...,4.0,883,NaN,19.0,15.6,NaN,15.8,16.4,16.2,16.5
7298,7298,2023-11-01 03:00,7299,108,서울,17.5,NaN,NaN,NaN,0.8,...,4.0,897,NaN,19.0,15.7,NaN,15.8,16.4,16.2,16.5


In [11]:
# 데이터프레임의 모든 컬럼 출력
with pd.option_context('display.max_columns', None):
    print(weather_data)

      Unnamed: 0                tm  rnum  stnId stnNm    ta  taQcflg  rn  \
0              0  2023-01-01 01:00     1    108    서울   1.5      NaN NaN   
1              1  2023-01-01 02:00     2    108    서울   1.5      NaN NaN   
2              2  2023-01-01 03:00     3    108    서울   1.6      NaN NaN   
3              3  2023-01-01 04:00     4    108    서울   1.5      NaN NaN   
4              4  2023-01-01 05:00     5    108    서울   0.8      NaN NaN   
...          ...               ...   ...    ...   ...   ...      ...  ..   
7295        7295  2023-11-01 00:00  7296    108    서울  17.4      NaN NaN   
7296        7296  2023-11-01 01:00  7297    108    서울  17.4      NaN NaN   
7297        7297  2023-11-01 02:00  7298    108    서울  17.6      NaN NaN   
7298        7298  2023-11-01 03:00  7299    108    서울  17.5      NaN NaN   
7299        7299  2023-11-01 04:00  7300    108    서울  17.3      NaN NaN   

      rnQcflg   ws  wsQcflg   wd  wdQcflg  hm  hmQcflg    pv    td      pa  \
0        

In [7]:
# 컬럼 정리하고 이름 바꾸기(전처리)
# 필요한 컬럼 tm(시간), ta(기온), rn(강수량), ws(풍속), wd(풍향), hm(습도)
weather_data.columns

Index(['Unnamed: 0', 'tm', 'rnum', 'stnId', 'stnNm', 'ta', 'taQcflg', 'rn',
       'rnQcflg', 'ws', 'wsQcflg', 'wd', 'wdQcflg', 'hm', 'hmQcflg', 'pv',
       'td', 'pa', 'paQcflg', 'ps', 'psQcflg', 'ss', 'ssQcflg', 'icsr', 'dsnw',
       'hr3Fhsc', 'dc10Tca', 'dc10LmcsCa', 'clfmAbbrCd', 'lcsCh', 'vs',
       'gndSttCd', 'dmstMtphNo', 'ts', 'tsQcflg', 'm005Te', 'm01Te', 'm02Te',
       'm03Te'],
      dtype='object')

In [12]:
weather_data.drop(columns=['rnum', 'stnId', 'stnNm', 'taQcflg',
       'rnQcflg', 'wsQcflg', 'wdQcflg', 'hmQcflg', 'pv',
       'td', 'pa', 'paQcflg', 'ps', 'psQcflg', 'ss', 'ssQcflg', 'icsr', 'dsnw',
       'hr3Fhsc', 'dc10Tca', 'dc10LmcsCa', 'clfmAbbrCd', 'lcsCh', 'vs',
       'gndSttCd', 'dmstMtphNo', 'ts', 'tsQcflg', 'm005Te', 'm01Te', 'm02Te',
       'm03Te'], inplace=True)

In [17]:
weather_data.drop(columns=['Unnamed: 0'], inplace=True)

In [15]:
weather_data.rename(columns={'tm': '시간','ta': '기온','rn': '강수량','ws': '풍속','wd': '풍향','hm': '습도'}, inplace=True)

In [18]:
weather_data

,시간,기온,강수량,풍속,풍향,습도
0,2023-01-01 01:00,1.5,NaN,1.9,250,71
1,2023-01-01 02:00,1.5,NaN,1.9,250,72
2,2023-01-01 03:00,1.6,NaN,1.6,250,74
3,2023-01-01 04:00,1.5,NaN,1.4,290,74
4,2023-01-01 05:00,0.8,NaN,2.4,290,62
...,...,...,...,...,...,...
7295,2023-11-01 00:00,17.4,NaN,1.0,160,80
7296,2023-11-01 01:00,17.4,NaN,1.3,200,81
7297,2023-11-01 02:00,17.6,NaN,2.9,180,83
7298,2023-11-01 03:00,17.5,NaN,0.8,140,82


* 두 데이터 합치기

In [88]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7300 entries, 0 to 7299
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시간      7300 non-null   object 
 1   기온      7300 non-null   float64
 2   강수량     981 non-null    float64
 3   풍속      7300 non-null   float64
 4   풍향      7300 non-null   int64  
 5   습도      7300 non-null   int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 342.3+ KB


In [89]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203206 entries, 0 to 203205
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   사용일자    203206 non-null  int64 
 1   노선명     203206 non-null  object
 2   역명      203206 non-null  object
 3   승차총승객수  203206 non-null  int64 
 4   하차총승객수  203206 non-null  int64 
 5   등록일자    203206 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 9.3+ MB


In [ ]:
# 기온에 따른 승하차 분석

In [ ]:
# 강수량에 따른 승하차 분석

In [ ]:
# 풍속 풍향에 따른 승하차 분석

In [ ]:
# 습도에 따른 승하차 분석